### HW lesson 6

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
items = pd.read_csv("items.csv")
print("Num unique items: {}\nNum unique categories: {}".format(items.shape[0], 
                                                                len(items['item_type'].unique())))
items.head(3)

Num unique items: 10237
Num unique categories: 10


,Name,item_category,item_brand,item_weight,item_type
0,зубная паста лакалют актив 75мл,"Красота, гигиена, бытовая химия",splat,75мл,зубная паста
1,зубная паста лакалют сенситив 75мл,"Красота, гигиена, бытовая химия",splat,75мл,зубная паста
2,зубная паста лесной бальзам ромашка и облепиха...,"Красота, гигиена, бытовая химия",лесной бальзам,NaN,зубная паста


In [3]:
purchases = pd.read_csv("purchases.csv")
print("Num unique users: {}".format(len(purchases['user_id'].unique())))
purchases.columns = ['user_id', 'Name']
purchases.head(3)

Num unique users: 32000


,user_id,Name
0,ed6b1aaf-21df-5b75-9b7f-ed67926cd17c,"шоколад ""alpen gold"" белый с миндалем и кокосо..."
1,ba82ad84-3a19-5a91-8e1e-7fd87628afb4,пюре тема говядина с гречкой с 8 месяцев
2,74a2856d-f0ec-59a6-89f3-1f80b294e852,колбаса микоян сервелат кремлевский варено-коп...


In [4]:
purchases = pd.merge(purchases, items)
purchases.head(3)

,user_id,Name,item_category,item_brand,item_weight,item_type
0,ed6b1aaf-21df-5b75-9b7f-ed67926cd17c,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
1,b4a10859-3f8c-5dc1-8d5d-5977f9aa8bde,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад
2,464053f2-ead4-500e-8486-9d5d66c1bbd7,"шоколад ""alpen gold"" белый с миндалем и кокосо...","Хлеб, сладости, снеки",alpen gold,90г,шоколад


In [5]:
len(purchases[purchases['item_type']=='шампунь']['user_id'].unique())

13224

In [6]:
purchases['y'] = purchases['item_type'].apply(lambda x: 1 if x=='шампунь' else 0, 1)

In [7]:
X = purchases.groupby(['user_id']).agg({'Name': lambda x: list(x), 
                                    'y': lambda x: max(x)})
X['user_id'] = [i for i in X.index.values]
X.columns = ['last_purchases', 'y', 'user_id']
X = X[['user_id', 'last_purchases', 'y']]
X.index = range(len(X))
X.head(3)

,user_id,last_purchases,y
0,00002f01-66e4-5ab8-8d1a-1562a4ddd418,[зубная паста splat stress off антистресс 75мл...,0
1,0000fed8-b063-51ef-8ca4-c42c5bd022ad,[шоколад schogetten black & white молочный с к...,0
2,0004cfe8-bcb2-5a2c-904b-643e0469cbe3,"[шоколад воздушный темный 85г, сыр белебеевски...",0


In [8]:
X['y'].value_counts()

0    18776
1    13224
Name: y, dtype: int64

In [9]:
X['last_purchases'] = X['last_purchases'].apply(lambda x: " ".join(x), 1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, X['y'], random_state=0)

In [11]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

pipeline = Pipeline([('last_purchases_selector', FeatureSelector(column='last_purchases')), 
                     ('last_purchases_tfidf', TfidfVectorizer()), 
                     ('clf', LogisticRegression())])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('last_purchases_selector',
                 FeatureSelector(column='last_purchases')),
                ('last_purchases_tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression())])

In [13]:
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.04267943, 0.05194219, 0.02460351, 0.95709101, 0.02702108,
       0.03198565, 0.01671249, 0.03525539, 0.06422554, 0.03025957])

In [14]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.226884, F-Score=1.000, Precision=1.000, Recall=1.000


In [15]:
X['y_pred'] = pipeline.predict_proba(X)[:, 1]
X = X.sort_values('y_pred', ascending=False)
X.head(3)

,user_id,last_purchases,y,y_pred
24541,c46ff21b-5115-56dc-88c7-654b91413323,шампунь oz! organiczone укрепление и питание 2...,1,1.0
19288,9a1d102d-41f6-5d99-a6e5-973586bbd95f,шампунь чистая линия сила 5 трав для всех типо...,1,1.0
704,056fe54e-a7e0-5f83-9e0c-4b3b673fe947,"шампунь head & shoulders против перхоти ""успок...",1,1.0


In [16]:
selected_users = X.iloc[:5000]['user_id'].values

In [17]:
purchases[purchases['user_id'].isin(selected_users)]['item_type'].value_counts(normalize=True)

шампунь         0.31952
колбаса         0.15796
зубная паста    0.11668
шоколад         0.09252
напиток         0.08260
корм            0.07628
кофе            0.06460
сыр             0.04500
пюре            0.03552
чай             0.00932
Name: item_type, dtype: float64

In [18]:
purchases[~purchases['user_id'].isin(selected_users)]['item_type'].value_counts(normalize=True)

колбаса         0.205126
зубная паста    0.167422
шоколад         0.125504
корм            0.119244
напиток         0.105430
кофе            0.084007
сыр             0.069126
шампунь         0.060948
пюре            0.053267
чай             0.009926
Name: item_type, dtype: float64